# Income Classification using a Linear Classifier

Modified from original code here: https://www.tensorflow.org/tutorials/wide

### Make the notebook compatible with both Python 2 and 3

http://python-future.org/compatible_idioms.html

In [15]:
from __future__ import absolute_import, division, print_function

In [16]:
import pandas as pd
from six.moves import urllib
import shutil
import tensorflow as tf

In [17]:
print(tf.__version__)
print(pd.__version__)

1.3.0
0.20.1


### Set up the file names where the training data and the test data are to be stored

Note that you'll have to manually create the "census" directory under the current working directory

In [18]:
TRAIN_FILE_NAME = "census/adult.data"
TEST_FILE_NAME = "census/adult.test"

### Download and store the training and test data from the UCI Machine Learning Repository

There are a whole host of interesting datasets here: https://archive.ics.uci.edu/ml/index.php

In [19]:
urllib.request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",
        TRAIN_FILE_NAME)

('census/adult.data', <httplib.HTTPMessage instance at 0x10ee8a368>)

In [20]:
urllib.request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.test",
        TEST_FILE_NAME)

('census/adult.test', <httplib.HTTPMessage instance at 0x10ee8a758>)

### The columns in the census data

In [21]:
CSV_COLUMNS = [
    "age", "workclass", "fnlwgt", "education", "education_num",
    "marital_status", "occupation", "relationship", "race", "gender",
    "capital_gain", "capital_loss", "hours_per_week", "native_country",
    "income_bracket"
]

### Read training data into a dataframe

Sample and explore the data to understand what information is available. This will also be used to set up feature columns which will serve as an input to our linear classifier.

In [22]:
df = pd.read_csv(
      TRAIN_FILE_NAME,
      names=CSV_COLUMNS,
      skipinitialspace=True,
      skiprows=1)

In [23]:
df.head()

age         workclass  fnlwgt  education  education_num  \
0   50  Self-emp-not-inc   83311  Bachelors             13   
1   38           Private  215646    HS-grad              9   
2   53           Private  234721       11th              7   
3   28           Private  338409  Bachelors             13   
4   37           Private  284582    Masters             14   

       marital_status         occupation   relationship   race  gender  \
0  Married-civ-spouse    Exec-managerial        Husband  White    Male   
1            Divorced  Handlers-cleaners  Not-in-family  White    Male   
2  Married-civ-spouse  Handlers-cleaners        Husband  Black    Male   
3  Married-civ-spouse     Prof-specialty           Wife  Black  Female   
4  Married-civ-spouse    Exec-managerial           Wife  White  Female   

   capital_gain  capital_loss  hours_per_week native_country income_bracket  
0             0             0              13  United-States          <=50K  
1             0             0              40  United-States          <=50K  
2             0             0              40  United-States          <=50K  
3             0             0              40           Cuba          <=50K  
4             0             0              40  United-States          <=50K

### Choose only those columns which seem relevant to predicting income

* Removed the "fnlwgt" column, the number of people the census takers believe that observation represents (sample weight)
* Removed "capital_gain" and "capital_loss", continuous, dense columns usually work well with neural networks

In [24]:
TRIMMED_REORDERED_COLUMNS = [
    "age", "workclass", "education", "education_num",
    "marital_status", "relationship", "race", "gender", "occupation", 
    "hours_per_week", "native_country", "income_bracket"
]

In [25]:
df = df[TRIMMED_REORDERED_COLUMNS]

In [26]:
df.head()

age         workclass  education  education_num      marital_status  \
0   50  Self-emp-not-inc  Bachelors             13  Married-civ-spouse   
1   38           Private    HS-grad              9            Divorced   
2   53           Private       11th              7  Married-civ-spouse   
3   28           Private  Bachelors             13  Married-civ-spouse   
4   37           Private    Masters             14  Married-civ-spouse   

    relationship   race  gender         occupation  hours_per_week  \
0        Husband  White    Male    Exec-managerial              13   
1  Not-in-family  White    Male  Handlers-cleaners              40   
2        Husband  Black    Male  Handlers-cleaners              40   
3           Wife  Black  Female     Prof-specialty              40   
4           Wife  White  Female    Exec-managerial              40   

  native_country income_bracket  
0  United-States          <=50K  
1  United-States          <=50K  
2  United-States          <=50K  
3           Cuba          <=50K  
4  United-States          <=50K

### Feature columns with categorical values

Find the unique values in a column and set up a categorical feature for those columns

In [27]:
df['gender'].unique()

array(['Male', 'Female'], dtype=object)

In [28]:
df['race'].unique()

array(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo',
       'Other'], dtype=object)

In [29]:
df['education'].unique()

array(['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
       'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
       '5th-6th', '10th', '1st-4th', 'Preschool', '12th'], dtype=object)

In [30]:
df['marital_status'].unique()

array(['Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
       'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'], dtype=object)

In [31]:
df['relationship'].unique()

array(['Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried',
       'Other-relative'], dtype=object)

In [32]:
df['workclass'].unique()

array(['Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'], dtype=object)

### Set up categorical feature columns

Use *tf.feature_column.categorical_column_with_vocabulary_list* if the categorical columns have a finite set of values that we know in advance

In [33]:
gender = tf.feature_column.categorical_column_with_vocabulary_list(
    "gender", ["Female", "Male"])

race = tf.feature_column.categorical_column_with_vocabulary_list(
    "race", ['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo', 'Other'])

education = tf.feature_column.categorical_column_with_vocabulary_list(
    "education", [
        "Bachelors", "HS-grad", "11th", "Masters", "9th",
        "Some-college", "Assoc-acdm", "Assoc-voc", "7th-8th",
        "Doctorate", "Prof-school", "5th-6th", "10th", "1st-4th",
        "Preschool", "12th"
    ])

marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
    "marital_status", [
        "Married-civ-spouse", "Divorced", "Married-spouse-absent",
        "Never-married", "Separated", "Married-AF-spouse", "Widowed"
    ])

relationship = tf.feature_column.categorical_column_with_vocabulary_list(
    "relationship", [
        "Husband", "Not-in-family", "Wife", "Own-child", "Unmarried",
        "Other-relative"
    ])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
    "workclass", [
        "Self-emp-not-inc", "Private", "State-gov", "Federal-gov",
        "Local-gov", "?", "Self-emp-inc", "Without-pay", "Never-worked"
    ])

### Columns with continuous values

Use *tf.feature_column.numeric_column* to set up columns which have values in a numeric range

In [34]:
age = tf.feature_column.numeric_column("age")

education_num = tf.feature_column.numeric_column("education_num")

hours_per_week = tf.feature_column.numeric_column("hours_per_week")

### Bucketed columns

Sometimes the relationship between a continuous feature and the label is not linear. A person's income may grow with age in the early stage of one's career, then the growth may slow at some point, and finally the income decreases after retirement. In this scenario, using the **raw age as a real-valued feature column** might not be a good choice because the model can only learn one of the three cases:

* Income always increases at some rate as age grows (positive correlation),
* Income always decreases at some rate as age grows (negative correlation), or
* Income stays the same no matter at what age (no correlation)

If we want to **learn the fine-grained correlation** between income and each age group separately, we can leverage bucketization. 

Bucketization is a process of dividing the entire range of a continuous feature into a set of consecutive bins/buckets, and then converting the original numerical feature into a bucket ID (as a categorical feature) depending on which bucket that value falls into. 

In [35]:
age_buckets = tf.feature_column.bucketized_column(
    age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

### Categorical column values might change over time

In [36]:
df['occupation'].unique()

array(['Exec-managerial', 'Handlers-cleaners', 'Prof-specialty',
       'Other-service', 'Adm-clerical', 'Sales', 'Craft-repair',
       'Transport-moving', 'Farming-fishing', 'Machine-op-inspct',
       'Tech-support', '?', 'Protective-serv', 'Armed-Forces',
       'Priv-house-serv'], dtype=object)

In [37]:
df['native_country'].unique()

array(['United-States', 'Cuba', 'Jamaica', 'India', '?', 'Mexico', 'South',
       'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany', 'Iran',
       'Philippines', 'Italy', 'Poland', 'Columbia', 'Cambodia',
       'Thailand', 'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala', 'China',
       'Japan', 'Yugoslavia', 'Peru', 'Outlying-US(Guam-USVI-etc)',
       'Scotland', 'Trinadad&Tobago', 'Greece', 'Nicaragua', 'Vietnam',
       'Hong', 'Ireland', 'Hungary', 'Holand-Netherlands'], dtype=object)

### Categorical columns with unknown values

If you don't know the list of categorical columns in advance then we use *tf.feature_column.categorical_column_with_hash_bucket* where every column value will be hashed to a unique integer.

The chances of collisions are usually low.

In [38]:
occupation = tf.feature_column.categorical_column_with_hash_bucket(
    "occupation", hash_bucket_size=1000)

native_country = tf.feature_column.categorical_column_with_hash_bucket(
    "native_country", hash_bucket_size=1000)

### Base columns which use the raw values from the dataset

In [142]:
base_columns = [
    gender, race, marital_status, workclass, occupation,
    native_country, age_buckets, education
]

### Crossed columns express more complex relationships between data

Some relationships between individual features and the output maybe hard to define. Two or more features considered together might have a more direct impact on the output. Feature crosses are **engineered features** which allows you to specify this more complex relationship.

Education and occupation when considered together will be a better predictor of income than either of them alone.

In [143]:
crossed_columns = [
    tf.feature_column.crossed_column(
        ["education", "occupation"], hash_bucket_size=1000),
    tf.feature_column.crossed_column(
        [age_buckets, "education", "occupation"], hash_bucket_size=1000),
    tf.feature_column.crossed_column(
        ["native_country", "occupation"], hash_bucket_size=1000)
]

### Continuous valued columns

If these columns are dense they are more suitable for deep neural networks

In [144]:
deep_columns = [
    education_num,
    hours_per_week
]

### The input function in an estimator maps the features and the corresponding labels

The standard library method *tf.estimator.inputs.pandas_input_fn* allows us to specify feature data as a pandas dataframe and the labels as a list.

The input function specifies the features and labels for training
    

In [145]:
def input_fn(file_name, num_epochs, shuffle):
  df = pd.read_csv(
      file_name,
      names=CSV_COLUMNS,
      skipinitialspace=True,
      skiprows=1)
  df = df[TRIMMED_REORDERED_COLUMNS]  
  
  # Remove NaN elements
  df = df.dropna(how="any", axis=0)

  # Use numeric labels to represent incomes below and above 50K
  labels = df["income_bracket"].apply(lambda x: ">50K" in x).astype(int)
  
  return tf.estimator.inputs.pandas_input_fn(
      x=df,
      y=labels,
      batch_size=100,
      num_epochs=num_epochs,
      shuffle=shuffle,
      num_threads=5)

In [146]:
MODEL_DIR = "./linear_classifier"

### Remove the old saved model so we generate entirely new parameters

In [147]:
shutil.rmtree(MODEL_DIR)

### Pass in all 3 sets of columns

For this model, the base columns are the ones which really affect the output

In [148]:
linear_estimator = tf.estimator.LinearClassifier(
        model_dir=MODEL_DIR, feature_columns=base_columns)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': './linear_classifier', '_save_summary_steps': 100}


In [149]:
linear_estimator.train(
      input_fn=input_fn(TRAIN_FILE_NAME, num_epochs=None, shuffle=True),
      steps=1000)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./linear_classifier/model.ckpt.
INFO:tensorflow:loss = 69.3147, step = 1
INFO:tensorflow:global_step/sec: 134.984
INFO:tensorflow:loss = 32.0356, step = 101 (0.742 sec)
INFO:tensorflow:global_step/sec: 147.143
INFO:tensorflow:loss = 33.1644, step = 201 (0.680 sec)
INFO:tensorflow:global_step/sec: 155.925
INFO:tensorflow:loss = 37.1448, step = 301 (0.642 sec)
INFO:tensorflow:global_step/sec: 151.583
INFO:tensorflow:loss = 28.4989, step = 401 (0.659 sec)
INFO:tensorflow:global_step/sec: 150.158
INFO:tensorflow:loss = 38.3806, step = 501 (0.666 sec)
INFO:tensorflow:global_step/sec: 150.409
INFO:tensorflow:loss = 43.2262, step = 601 (0.665 sec)
INFO:tensorflow:global_step/sec: 153.754
INFO:tensorflow:loss = 36.5726, step = 701 (0.651 sec)
INFO:tensorflow:global_step/sec: 152.791
INFO:tensorflow:loss = 39.4534, step = 801 (0.654 sec)
INFO:tensorflow:global_step/sec: 152.261
INFO:tensorflow:loss = 31.41

### Evaluate the test data and predict the income levels of the adults

In [150]:
results = linear_estimator.evaluate(
      input_fn=input_fn(TEST_FILE_NAME, num_epochs=1, shuffle=False),
      steps=None)

INFO:tensorflow:Starting evaluation at 2017-09-08-03:16:41
INFO:tensorflow:Restoring parameters from ./linear_classifier/model.ckpt-1000
INFO:tensorflow:Finished evaluation at 2017-09-08-03:16:49
INFO:tensorflow:Saving dict for global step 1000: accuracy = 0.838216, accuracy_baseline = 0.763774, auc = 0.88621, auc_precision_recall = 0.696221, average_loss = 0.348924, global_step = 1000, label/mean = 0.236226, loss = 34.8517, prediction/mean = 0.224601


In [151]:
for key in sorted(results):
    print("%s: %s" % (key, results[key]))

accuracy: 0.838216
accuracy_baseline: 0.763774
auc: 0.88621
auc_precision_recall: 0.696221
average_loss: 0.348924
global_step: 1000
label/mean: 0.236226
loss: 34.8517
prediction/mean: 0.224601
